## Load libraries

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

## List with the url address of each recipe from www.przepisy.pl

In [3]:
recipe_url='https://www.przepisy.pl/przepisy/szukaj/*'

In [4]:
recipe_url_bs = BeautifulSoup(requests.get(recipe_url).text, 'lxml')

In [5]:
recipe_url_page=recipe_url_bs.select('app-pagination a.pagination__btn')

In [7]:
recipe_url_pages=[element.text.strip() for element in recipe_url_page]
recipe_url_pages=[int(element) for element in recipe_url_pages if element!='']
page_number=max(recipe_url_pages)

In [10]:
recipes = []

for i in range(1,page_number-1):
        recipe_url='https://www.przepisy.pl/przepisy/posilek/dania-do-pracy?page='+str(i)
        bs_lunch = BeautifulSoup(requests.get(recipe_url).text, 'lxml')
        recipe_lu=bs_lunch.select('div.title-recipe a.recipe-box__title')
        for recipe in recipe_lu:
            recipe_address = recipe['href']
            recipes.append( recipe_address )

## Matrix with ingredients

Structure created based on code: https://github.com/dataworkshop/dw-cracow-project/blob/master/personal/PawelDulak/recipes_shopping_list/ingrediends_scraping/scraping_przepisy_pl.ipynb

In [11]:
ingredients = []
for recipe in recipes:
    bs = BeautifulSoup(requests.get(recipe).text, 'lxml')
    ingredients_ul = bs.select('ul.ingredient-ul li.ingredient-li')
    for ingredient_li in ingredients_ul:
        ingredient_name = ingredient_li.select_one('div.ingredient-name input').get('id')
        ingredient_quantity_tag = ingredient_li.select_one('span.quantity')
        if ingredient_quantity_tag:
            ingredient_quantity = ingredient_quantity_tag.text
        else: 
            ingredient_quantity = '---'
    
        ingredients.append( [recipe, ingredient_name, ingredient_quantity] )

## Save matrix to CSV file

In [13]:
ingredients_df = pd.DataFrame(ingredients, columns=['URL','nazwa_skladnika','ilosc'])
ingredients_df.to_csv('ingredients.csv', index=False, header=True)